# Linea de Muerte, Tercera Competencia

### Legal Notice

Alumnos, de la comision del lunes o del jueves

que terminen en el Private Leaderboard por debajo de la ganancia  del submit que genera este muy basico script

tendran severamente comprometidas sus chances de aprobar la asignatura

por favor notar que este script construye todo de cero, y unicamente utiliza conceptos ya que estaban disponibles an la Primera Competencia

### Requerimientos

* Este script necesita apenas una virtual machine con **32 GB** de memoria RAM
* Corre en una hora y cuarenta

## Inicializacion

Establezco Experimento

In [1]:
# limpio la memoria
format(Sys.time(), "%a %b %d %X %Y")
rm(list = ls(all.names = TRUE)) # remove all objects
gc(full = TRUE, verbose= FALSE) # garbage collection

dir.create("~/buckets/b1/exp/lineademuerte/", showWarnings = FALSE)
setwd( "~/buckets/b1/exp/lineademuerte/" )

[1] "Sun Dec 01 05:47:27 2024"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,655723,35.1,1446040,77.3,1173292,62.7
Vcells,1219535,9.4,8388608,64.0,1924982,14.7


## Creacion clase_ternaria

Creacion de clase_ternaria a partir del dataset crudo

In [2]:
require( "data.table" )

# leo el dataset
dataset <- fread("~/buckets/b1/datasets/competencia_03_crudo.csv.gz" )

# calculo el periodo0 consecutivo
setorder( dataset, numero_de_cliente, foto_mes )
dataset[, periodo0 := as.integer(foto_mes/100)*12 +  foto_mes%%100]

# calculo topes
periodo_ultimo <- dataset[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1

# calculo los leads de orden 1 y 2
dataset[, c("periodo1", "periodo2") :=
    shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente ]

# assign most common class values = "CONTINUA"
dataset[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dataset[ periodo0 < periodo_ultimo &
    ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
    clase_ternaria := "BAJA+1" ]

# calculo BAJA+2
dataset[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
    & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
    clase_ternaria := "BAJA+2" ]

dataset[, c("periodo0", "periodo1", "periodo2") := NULL ]

Loading required package: data.table



Verificacion de clase_ternaria

In [3]:
tbl <- dataset[, .N, list(foto_mes, clase_ternaria)]
setorder(tbl, foto_mes, clase_ternaria)
tbl

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,688
201901,BAJA+2,718
201901,CONTINUA,123346
201902,BAJA+1,722
201902,BAJA+2,688
201902,CONTINUA,124389
201903,BAJA+1,689
201903,BAJA+2,760
201903,CONTINUA,124987


## Preprocesamiento

Solamente se hará un Feature Engineering Histórico

Intencionalmente **NO** se hacen
* Data Quality = Catastrophe Analysis   ( variables rotas que están en CERO )
* Data Drifting,  ajuste por Inflacion
* Feature Engineering Manual intra-mes
* Feature Engineering de nuevas variables a partir de hojas de Random Forest

### Feature Engineering Historico

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1    valor actual - lag1
* lag2  lags de orden 2
* delta2  valor actual - lag2

se pasa de  155 campos a 763

(*) Excepto para los campos  [ numero_de_cliente,  foto_mes,  clase_ternaria]

In [4]:
# Feature Engineering Historico
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )


dataset[, 
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

dataset[, 
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags de orden 1
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Veriricacion de los campos recien creados

In [5]:
ncol(dataset)
colnames(dataset)

[1] 763

[1] "numero_de_cliente"                          
  [2] "foto_mes"                                   
  [3] "active_quarter"                             
  [4] "cliente_vip"                                
  [5] "internet"                                   
  [6] "cliente_edad"                               
  [7] "cliente_antiguedad"                         
  [8] "mrentabilidad"                              
  [9] "mrentabilidad_annual"                       
 [10] "mcomisiones"                                
 [11] "mactivos_margen"                            
 [12] "mpasivos_margen"                            
 [13] "cproductos"                                 
 [14] "tcuentas"                                   
 [15] "ccuenta_corriente"                          
 [16] "mcuenta_corriente_adicional"                
 [17] "mcuenta_corriente"                          
 [18] "ccaja_ahorro"                               
 [19] "mcaja_ahorro"                               
 [20] "mcaja_ahorro_adicional"                     
 [21] "mcaja_ahorro_dolares"                       
 [22] "cdescubierto_preacordado"                   
 [23] "mcuentas_saldo"                             
 [24] "ctarjeta_debito"                            
 [25] "ctarjeta_debito_transacciones"              
 [26] "mautoservicio"                              
 [27] "ctarjeta_visa"                              
 [28] "ctarjeta_visa_transacciones"                
 [29] "mtarjeta_visa_consumo"                      
 [30] "ctarjeta_master"                            
 [31] "ctarjeta_master_transacciones"              
 [32] "mtarjeta_master_consumo"                    
 [33] "cprestamos_personales"                      
 [34] "mprestamos_personales"                      
 [35] "cprestamos_prendarios"                      
 [36] "mprestamos_prendarios"                      
 [37] "cprestamos_hipotecarios"                    
 [38] "mprestamos_hipotecarios"                    
 [39] "cplazo_fijo"                                
 [40] "mplazo_fijo_dolares"                        
 [41] "mplazo_fijo_pesos"                          
 [42] "cinversion1"                                
 [43] "minversion1_pesos"                          
 [44] "minversion1_dolares"                        
 [45] "cinversion2"                                
 [46] "minversion2"                                
 [47] "cseguro_vida"                               
 [48] "cseguro_auto"                               
 [49] "cseguro_vivienda"                           
 [50] "cseguro_accidentes_personales"              
 [51] "ccaja_seguridad"                            
 [52] "cpayroll_trx"                               
 [53] "mpayroll"                                   
 [54] "mpayroll2"                                  
 [55] "cpayroll2_trx"                              
 [56] "ccuenta_debitos_automaticos"                
 [57] "mcuenta_debitos_automaticos"                
 [58] "ctarjeta_visa_debitos_automaticos"          
 [59] "mttarjeta_visa_debitos_automaticos"         
 [60] "ctarjeta_master_debitos_automaticos"        
 [61] "mttarjeta_master_debitos_automaticos"       
 [62] "cpagodeservicios"                           
 [63] "mpagodeservicios"                           
 [64] "cpagomiscuentas"                            
 [65] "mpagomiscuentas"                            
 [66] "ccajeros_propios_descuentos"                
 [67] "mcajeros_propios_descuentos"                
 [68] "ctarjeta_visa_descuentos"                   
 [69] "mtarjeta_visa_descuentos"                   
 [70] "ctarjeta_master_descuentos"                 
 [71] "mtarjeta_master_descuentos"                 
 [72] "ccomisiones_mantenimiento"                  
 [73] "mcomisiones_mantenimiento"                  
 [74] "ccomisiones_otras"                          
 [75] "mcomisiones_otras"                          
 [76] "cforex"                                     
 [77] "cforex_buy"                                

## Modelado

### Training Strategy

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 201901, 202107 ]  con un undersampling de los CONTINUA del 2% por un tema de velocidad

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling 
   * training = [ 201901, 202106 ]  con un undersampling de los CONTINUA del 2%  por un tema de velocidad

In [6]:
GLOBAL_semilla <- 102191

In [7]:
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria"))
)

set.seed(GLOBAL_semilla, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

dfuture <- dataset[foto_mes==202109]

# undersampling de los CONTINIA al 8%
dataset[, fold_train :=  foto_mes<= 202107 &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < 0.02 ) ]

dataset[, clase01 := ifelse( clase_ternaria=="CONTINUA", 0, 1 )]

require("lightgbm")

# dejo los datos en el formato que necesita LightGBM
dvalidate <- lgb.Dataset(
  data = data.matrix(dataset[foto_mes==202107, campos_buenos, with = FALSE]),
  label = dataset[foto_mes==202107, clase01],
  free_raw_data = TRUE
)

# aqui se hace la magia informatica con los pesos para poder reutilizar
#  el mismo dataset para training y final_train
dtrain <- lgb.Dataset(
  data = data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label = dataset[fold_train == TRUE, clase01],
  weight = dataset[fold_train == TRUE, ifelse( foto_mes<=202106, 1.0, 0.0)],
  free_raw_data = TRUE
)

rm( dataset )
gc(full = TRUE, verbose= FALSE) # garbage collection

Loading required package: lightgbm



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1836210,98.1,3502474,187.1,2591545,138.5
Vcells,324281750,2474.1,3645902481,27816.1,3589568062,27386.3


verificacion de datasets de entrenamiento

In [8]:
nrow( dfuture )
nrow( dvalidate )
nrow( dtrain )

[1] 165644

[1] 165152

[1] 132541

### Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve 

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones intelifgentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 1024]
  * min_data_in_leaf  [64, 8196]

In [9]:

# parametros basicos del LightGBM
param_basicos <- list(
    objective = "binary",
    metric = "auc",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    force_row_wise = TRUE, # para evitar warning
    seed = GLOBAL_semilla,
    max_bin = 31,
    learning_rate = 0.03,
    feature_fraction = 0.5
)


EstimarGanancia_AUC_lightgbm <- function(x) {

    message(format(Sys.time(), "%a %b %d %X %Y"))
    param_train <- list(
      num_iterations = 2048, # valor grande, lo limita early_stopping_rounds
      early_stopping_rounds = 200
    )

    param_completo <- c(param_basicos, param_train, x)

    modelo_train <- lgb.train(
      data = dtrain,
      valids = list(valid = dvalidate),
      eval = "auc", 
      param = param_completo,
      verbose = -100
    )

    AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

    # esta es la forma de devolver un parametro extra
    attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)
    
    rm(modelo_train)
    gc(full= TRUE, verbose= FALSE)
    
    return(AUC)
}


Complejo seteo de la Bayesian Optimization

In [10]:
# paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames




el seteo propiamente

In [11]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn = EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize = FALSE, # estoy Maximizando AUC
    noisy = FALSE,
    par.set = makeParamSet(
       makeIntegerParam("num_leaves", lower = 8L, upper = 1024L),
       makeIntegerParam("min_data_in_leaf", lower = 64L, upper = 8192L)
    ),
    has.simple.signature = FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time = 600,
    save.file.path = "lineademuerte.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters = 10  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type = "se",
    covtype = "matern3_2",
    control = list(trace = TRUE)
)


Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado

In [12]:
bayesiana_salida <- mbo(obj.fun, learner = surr.km, control = ctrl)

Computing y column(s) for design. Not provided.

Sun Dec 01 05:50:04 2024

Sun Dec 01 05:53:31 2024

Sun Dec 01 05:56:52 2024

Sun Dec 01 06:01:40 2024

Sun Dec 01 06:04:55 2024

Sun Dec 01 06:08:13 2024

Sun Dec 01 06:11:43 2024

Sun Dec 01 06:15:16 2024

[mbo] 0: num_leaves=689; min_data_in_leaf=5463 : y = 0.938 : 207.6 secs : initdesign

[mbo] 0: num_leaves=436; min_data_in_leaf=4092 : y = 0.939 : 201.0 secs : initdesign

[mbo] 0: num_leaves=319; min_data_in_leaf=377 : y = 0.944 : 287.9 secs : initdesign

[mbo] 0: num_leaves=854; min_data_in_leaf=6597 : y = 0.938 : 194.3 secs : initdesign

[mbo] 0: num_leaves=537; min_data_in_leaf=4552 : y = 0.939 : 198.6 secs : initdesign

[mbo] 0: num_leaves=154; min_data_in_leaf=1252 : y = 0.941 : 209.7 secs : initdesign

[mbo] 0: num_leaves=933; min_data_in_leaf=2450 : y = 0.94 : 213.2 secs : initdesign

[mbo] 0: num_leaves=69; min_data_in_leaf=7618 : y = 0.937 : 193.3 secs : initdesign

Saved the current state after iteration 1 in the file line

obtengo los mejores hiperparametros

In [17]:
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
setorder(tb_bayesiana, -y, -num_iterations) # ordeno en forma descendente por AUC = y
mejores_hiperparametros <- tb_bayesiana[1, # el primero es el de mejor AUC
    list(num_leaves, min_data_in_leaf, num_iterations)]

print(mejores_hiperparametros)

   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:        956               64           1995


## Produccion

### Final Model

magia informatica, ahora  dtrain es [201901, 202107] a todos les asigno el peso en 1.0

In [18]:
set_field(dtrain, "weight", rep(1.0, nrow(dtrain)))

In [19]:
param_final <- c(param_basicos, mejores_hiperparametros)

Genero el modelo final

In [20]:
final_model <- lgb.train(
    data = dtrain,
    param = param_final,
    verbose = -100
)

### Scoring

Aplico el modelo final a los datos del futuro

In [21]:
prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with = FALSE])
)

### Kaggle

Genero la salida para Kaggle

In [22]:
# genero la tabla de entrega
tb_entrega <- dfuture[, list(numero_de_cliente)]
tb_entrega[, prob := prediccion]

# ordeno por probabilidad descendente
setorder(tb_entrega, -prob)
tb_entrega[, prob := NULL] # ya no necesito prob
tb_entrega[, Predicted := 0L]
tb_entrega[1:11000, Predicted := 1L]

fwrite(tb_entrega, file = "lineademuerte_11000.csv" )

In [23]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Dec 01 07:23:56 2024"